In [3]:
import xml.etree.ElementTree as ET
import xml.dom.minidom as dm
from lxml import etree
import lxml
from bs4 import BeautifulSoup

In [10]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# xml文件读取

In [4]:
xl_path = r".\testfile\test.xml"

In [9]:
ET_tree = ET.ElementTree(file=xl_path)  #方法一：无xpath功能
Parse_tree = ET.parse(xl_path)          #方法二：同方法一
EE_tree = etree.parse(xl_path)          #方法三：有xpath
DM_tree = dm.parse(xl_path)             #方法四：简单的方法

# xml文件解析

## 方法一：ElementTree

In [34]:
ET_root = ET_tree.getroot()  #获取树的根,根也是元素,元素就可以有属性
ET_root.tag
ET_root.attrib
ET_root.attrib['school']

'bookstore'

{'school': 'AutoAcademy'}

'AutoAcademy'

### 用遍历的方法一级一级进行查找目标内容

#### 普通一级一级的遍历

In [16]:
for child in ET_root:
    child_tag = child.tag
    child_attrib = child.attrib['id']             #获取子元素中的属性或者属性的值(如果有该属性)
    text = child.text                               #获取元素中的内容
    print(child_tag,child_attrib,text,sep='\n')
    for sub_child in child:
        print(sub_child.tag,sub_child.attrib)

history
shelf001
 
        text,source 
        
book {'name': 'xxxx10', 'country': 'Russia'}
book {'name': 'xxxx20', 'country': 'China'}
book {'region': 'China'}
documentation {'country': 'China'}
magzine {'date': '2020-12', 'country': 'China'}
tag {'location': 'head'}
tag {}
tag {'location': 'tail'}
political
shelf002
 
        
book {'name': 'subrelease01', 'country': 'China'}
book {'name': 'subrelease02', 'id': 'U', 'country': 'America'}
book {'name': 'subrelease03', 'country': 'China'}
book {'name': 'subrelease03', 'country': 'Japan'}
magzine {'name': 'Global'}
nature
shelf002

        
book {'name': 'World', 'price': '50'}
science
Shelf003

        
book {'name': 'Global', 'price': '100'}
history
shelf005

        Second history value
        
book {'name': 'NAN', 'country': 'America'}
book {'name': 'South', 'country': 'Korea'}


#### 还可以通过索引值来访问特定的元素

In [19]:
ET_root[1].tag
ET_root[1].attrib

'political'

{'id': 'shelf002', 'name': 'release01', 'hash': 'f200013e'}

#### 对指定的条件进行遍历

In [37]:
for elem in ET_tree.iter(tag='history'):   #这里指的是根目录下所有级别里的标签，不仅仅只是下一级
    elem.tag
    elem.attrib
    elem.text

'history'

{'id': 'shelf001', 'type': 'Recent', 'hash': '1cdf045c'}

' \n        text,source \n        '

'history'

{'id': 'shelf005', 'type': 'Antique'}

'\n        Second history value\n        '

#### 也可以对指定位置的元素进行遍历

In [49]:
for elem in ET_root[1][1]:
    print(elem.tag,elem.attrib)

yunxiao {'local': 'XiangGang', 'price': '200'}
yiyun {'local': 'America', 'price': '30'}
shidai {'local': 'MainLand', 'price': '150'}
price {}


### findall:以列表形式返回所有匹配的子元素
    其它：
        find:
        iterfind:

#### '.':选取根节点

In [55]:
ET_tree.findall(r'.')[0].tag   #注意它返回的是列表

'bookstore'

#### ‘/’:选取子节点

#### '//'选取当前节点下所有元素，部分级别

In [ ]:
ET_tree.findall(r'./history//')      #查找所有
ET_tree.findall(r'./history//book')  #查找所有book

#### '[]':指定位置，和列表不同的是，它只能是：1,2,3...或last(),last()-1

In [60]:
for elem in ET_tree.findall(r'./history[1]//book'):  #查找第一个history下面的所有book
    elem.tag

'book'

'book'

'book'

In [68]:
for elem in ET_tree.findall(r'./history[last()]//book'):
    elem.tag

'book'

'book'

#### ‘@’：选取属性

In [57]:
for elem in ET_tree.findall(r'./political/book[@country]'):   #所有带有contry属性的元素
    elem.attrib

{'name': 'subrelease01', 'country': 'China'}

{'name': 'subrelease02', 'id': 'U', 'country': 'America'}

{'name': 'subrelease03', 'country': 'China'}

{'name': 'subrelease03', 'country': 'Japan'}

In [58]:
for elem in ET_tree.findall(r'./political/book[@country="China"]'):   #所有带有contry属性的元素,且值为‘China’
    elem.attrib

{'name': 'subrelease01', 'country': 'China'}

{'name': 'subrelease03', 'country': 'China'}

## xpath:方法查找

### '.':选取当前节点

In [70]:
EE_tree.xpath(r'.')[0].tag

'bookstore'

### './':从根节点选取

In [71]:
EE_tree.xpath(r'/bookstore/*')        #选取根节点(bookstore)下面所有的子节点
EE_tree.xpath(r'./*')                 #同上一条
EE_tree.xpath(r'/bookstore/history')    #选取bookstore下面的所有的history节点

[<Element history at 0x1035cd388c0>,
 <Element political at 0x1035ce28b80>,
 <Element nature at 0x1035ce28880>,
 <Element science at 0x1035d09ec40>,
 <Element history at 0x1035d09e300>]

[<Element history at 0x1035cd388c0>,
 <Element political at 0x1035ce28b80>,
 <Element nature at 0x1035ce28880>,
 <Element science at 0x1035d09ec40>,
 <Element history at 0x1035d09e300>]

[<Element history at 0x1035cd388c0>, <Element history at 0x1035d09e300>]

### '//':在所有层级里查找

### '..':选取当前节点的父节点

### '[]':指定条件查找

### "contains":包含'xxx'

In [73]:
for elem in EE_tree.xpath(r'.//book[contains(@country,"i")]'):
    elem.attrib

{'name': 'xxxx10', 'country': 'Russia'}

{'name': 'xxxx20', 'country': 'China'}

{'name': 'subrelease01', 'country': 'China'}

{'name': 'subrelease02', 'id': 'U', 'country': 'America'}

{'name': 'subrelease03', 'country': 'China'}

{'name': 'NAN', 'country': 'America'}

### 'starts-with':开头是‘xxx’

In [77]:
for elem in EE_tree.xpath(r'.//book[starts-with(@country,"R")]'):
    elem.attrib

{'name': 'xxxx10', 'country': 'Russia'}

### 运算符定位：
    +,-,*,div,=(等于),!=,<,>,<=,>=,or,and,mod(计算除法余数)

### '*':匹配所有

### '|':选取若干路径

### '@':选取属性

In [83]:
EE_tree.xpath(r'./history/@*')

['shelf001', 'Recent', '1cdf045c', 'shelf005', 'Antique']

In [84]:
EE_tree.xpath(r'.//book[1]/@country')

['Russia', 'China', 'America']

### text():取文本

In [87]:
EE_tree.xpath(r'.//book[1]/text()')

['\n            this is first history 1-1....price:40RMB\n            ',
 '\n        ',
 ' \n            ',
 '\n            ',
 '\n            ',
 '\n            ',
 '\n            xml,sgml\n            \n        ',
 '\n            test warning\n        ',
 '\n            This is second history 2-1,price:15RMB\n            ',
 '\n        ']

## bs4:解析xml

In [88]:
soup = BeautifulSoup(open(xl_path),'lxml')         #注意:要先打开xml文件

### 基本方法：

In [90]:
soup.book   #按标签名查找，返回第一个

<book country="Russia" name="xxxx10">
            this is first history 1-1....price:40RMB
            <price>40</price>
</book>

In [91]:
soup.book.attrs   #返回找到的元素的属性

{'name': 'xxxx10', 'country': 'Russia'}

In [93]:
soup.book['country']   #直接返回属性的值

'Russia'

### find_all:

In [95]:
soup.find_all()

[<html><body><bookstore school="AutoAcademy"> 
     This is the bookstore value...
     <history hash="1cdf045c" id="shelf001" type="Recent"> 
         text,source 
         <book country="Russia" name="xxxx10">
             this is first history 1-1....price:40RMB
             <price>40</price>
 </book>
 <book country="China" name="xxxx20">
             this is first history 1-2...price:30RMB
             <illustration author="Mr Dao"></illustration>
 <price>30</price>
 </book>
 <book region="China">
             this is first history 1-3,price:9.8RMB
             <price>9.8</price>
 </book>
 <documentation country="China"></documentation>
 <magzine country="China" date="2020-12">
             Brief description:......xxxx10
             <price>12</price>
 </magzine>
 <tag location="head">cue word head</tag>
 <tag>cue word middle</tag>
 <tag location="tail">cue word tail</tag>
 </history>
 <political hash="f200013e" id="shelf002" name="release01">
 <book country="China" name="subreleas

In [96]:
soup.find_all('book')

[<book country="Russia" name="xxxx10">
             this is first history 1-1....price:40RMB
             <price>40</price>
 </book>,
 <book country="China" name="xxxx20">
             this is first history 1-2...price:30RMB
             <illustration author="Mr Dao"></illustration>
 <price>30</price>
 </book>,
 <book region="China">
             this is first history 1-3,price:9.8RMB
             <price>9.8</price>
 </book>,
 <book country="China" name="subrelease01">
 <aiqing local="XiangGang"></aiqing>
 <kongbu local="America"></kongbu>
 <zhanzheng local="MainLand"></zhanzheng>
 <price>100</price>
             xml,sgml
             
         </book>,
 <book country="America" id="U" name="subrelease02">
 <yunxiao local="XiangGang" price="200">
 </yunxiao>
 <yiyun local="America" price="30">
 </yiyun>
 <shidai local="MainLand" price="150">
 </shidai>
             xml2,sgml2 
             <price>10</price>
 </book>,
 <book country="China" name="subrelease03">
 <price>5</price>
 </book>

In [97]:
soup.find_all('book',country="China")

[<book country="China" name="xxxx20">
             this is first history 1-2...price:30RMB
             <illustration author="Mr Dao"></illustration>
 <price>30</price>
 </book>,
 <book country="China" name="subrelease01">
 <aiqing local="XiangGang"></aiqing>
 <kongbu local="America"></kongbu>
 <zhanzheng local="MainLand"></zhanzheng>
 <price>100</price>
             xml,sgml
             
         </book>,
 <book country="China" name="subrelease03">
 <price>5</price>
 </book>]

In [99]:
soup.find_all('book',country="China",limit=1)

[<book country="China" name="xxxx20">
             this is first history 1-2...price:30RMB
             <illustration author="Mr Dao"></illustration>
 <price>30</price>
 </book>]

In [100]:
soup.find_all(['book','magzine'])  #同时查找多个标签的所有

[<book country="Russia" name="xxxx10">
             this is first history 1-1....price:40RMB
             <price>40</price>
 </book>,
 <book country="China" name="xxxx20">
             this is first history 1-2...price:30RMB
             <illustration author="Mr Dao"></illustration>
 <price>30</price>
 </book>,
 <book region="China">
             this is first history 1-3,price:9.8RMB
             <price>9.8</price>
 </book>,
 <magzine country="China" date="2020-12">
             Brief description:......xxxx10
             <price>12</price>
 </magzine>,
 <book country="China" name="subrelease01">
 <aiqing local="XiangGang"></aiqing>
 <kongbu local="America"></kongbu>
 <zhanzheng local="MainLand"></zhanzheng>
 <price>100</price>
             xml,sgml
             
         </book>,
 <book country="America" id="U" name="subrelease02">
 <yunxiao local="XiangGang" price="200">
 </yunxiao>
 <yiyun local="America" price="30">
 </yiyun>
 <shidai local="MainLand" price="150">
 </shidai>
      

In [101]:
soup.find_all(['book','magzine'])[0].find_all(),   #同样所有的<class 'bs4.element.Tag'>,find_all都是适用的

([<price>40</price>],)